# Разные задачки

# Продолжаем про декораторы 

## Замена исходной функции

Функция, возвращаемая декоратором, заменяет декорируемую

In [136]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

In [139]:
def target():
    print('running target()')

In [140]:
@deco
def target():
    print('running target()')

In [141]:
target()

running inner()


In [143]:
target  # now reference to inner 

<function __main__.deco.<locals>.inner()>

## Running at import time

Key feature декораторов в том, что они применяются сразу после объявления декорируемой функции во время загрузки модуля в Python 

In [147]:
registry = []

def register(func):
    print(f'running register {func}')
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    
@register
def f2():
    print('running f2()')

def f3():
    print('running f3()')
    
def run():
    print('running run()')
    print('registry', registry)
    f1()
    f2()
    f3()

running register <function f1 at 0x7f6c72626f80>
running register <function f2 at 0x7f6c72626ef0>


In [148]:
run()

running run()
registry [<function f1 at 0x7f6c72626f80>, <function f2 at 0x7f6c72626ef0>]
running f1()
running f2()
running f3()


## Декоратор-логгер

### logging

 Модуль `logging` содержит набор функций для логирования различного поведения в вашей программе. Например, для понимания, что ваша программа работает как ожидается, или там идет что-то не так

Есть разные уровни логирования: debug, info, warning, error, critical

In [1]:
import logging

In [2]:
logging.info('Working as expected')
logging.warning('Smth might go wrong, check this!')

Можно логировать в файл (надо начать новую сессию в Python)

In [15]:
import os
import logging

log_filename = os.path.join('data', 'example.log')
log_level = logging.DEBUG

logging.basicConfig(
    filename=log_filename,
    encoding='utf-8',
    level=log_level,
    format='%(asctime)s %(message)s'  # add time
)

logging.debug('One very thorough message to save in our log')
logging.info('Just some info, also in log')
logging.warning('Something might be wrong')
logging.error('Logging our failure')

In [2]:
! cat data/example.log

2022-09-29 03:04:13,025 One very thorough message to save in our log
2022-09-29 03:04:13,025 Just some info, also in log
2022-09-29 03:04:13,025 Something might be wrong
2022-09-29 03:04:13,025 Logging our failure


### Напишем декоратор

In [1]:
import logging
from functools import wraps, partial

Стоит посмотреть в сторону объявления отдельного объекта логгера с помощью getLogger

In [23]:
@logged(log_filename='test_log.log')
def add(x, y):
    return x + y

add(1, 2)

3

In [25]:
import time

In [26]:
@logged
@clock(active=True)
def add(x, y):
    time.sleep(2)
    return x + y

In [27]:
add(10, 10**10)

add((10, 10000000000)) -> 10000000010 executed in 2.00s


10000000010

## Декоратор -- assert типов (2 балла)

Мы уже говорили про type hints, а также про то, что никаких ошибок без использования сторонних модулей некорректные типы нам не дадут. Давайте реализуем логику проверки типов с помощью декоратора.

Типы могут быть заданы как стандартными названиями, так и с использованыем модуля typing. Для Any пропускайте проверку

In [11]:
from typing import Any

In [9]:
@check_types
def func(a: int, b: str) -> str:
    return a * b

Стоит выбросить ValueError, если есть несовпадение между реальным и ожидаемым типом у какой-то из переданных переменных или у возвращаемого значения